# Constrained Lists

Recall the example we had in the last video:

In [1]:
from pydantic import BaseModel, PositiveInt, ValidationError

class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: PositiveInt = 1

Our `center` point can only be defined a a point in 2D space.

But what if we wanted to allow specifying a 2D point (2 element coordinate), or a 3D point (3 element coordinate)?

One approach might be this:

In [2]:
class Sphere(BaseModel):
    center: tuple[int, int] | tuple[int, int, int] = (0, 0)
    radius: PositiveInt = 1

Then we can define a 2D sphere (a circle, so choice of model name is not ideal):

In [3]:
Sphere(center=(1, 1), radius=10)

Sphere(center=(1, 1), radius=10)

or a 3D sphere:

In [4]:
Sphere(center=(1, 2, 3), radius=5)

Sphere(center=(1, 2, 3), radius=5)

If we try to specify a 4D tuple, we'll get an exception:

In [5]:
try:
    Sphere(center=(1, 2, 3, 4))
except ValidationError as ex:
    print(ex)

2 validation errors for Sphere
center.tuple[int, int]
  Tuple should have at most 2 items after validation, not 4 [type=too_long, input_value=(1, 2, 3, 4), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long
center.tuple[int, int, int]
  Tuple should have at most 3 items after validation, not 4 [type=too_long, input_value=(1, 2, 3, 4), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


So, we get an exception, but the error reporting is a bit funky and not very easy to even understand.

For now, let's ignore the fact that we specified we wanted a tuple, and suppose we are OK with a list instead.

Then we can use Pydantic's constrained list, to contrains our list to a minum of 2 elements, and a maximum of 3 elements:

In [6]:
from pydantic import conlist 


class Sphere(BaseModel):
    center: conlist(int, min_length=2, max_length=3) = [0, 0]
    radius: PositiveInt = 1

In [7]:
Sphere(center=(1, 2))

Sphere(center=[1, 2], radius=1)

In [8]:
Sphere(center=[1, 2, 3])

Sphere(center=[1, 2, 3], radius=1)

And let's see what the exception looks like if we do not have the required number of elements for `center`:

In [9]:
try:
    Sphere(center=(1, 2, 3, 4))
except ValidationError as ex:
    print(ex)

1 validation error for Sphere
center
  List should have at most 3 items after validation, not 4 [type=too_long, input_value=(1, 2, 3, 4), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


In [10]:
try:
    Sphere(center=(1, ))
except ValidationError as ex:
    print(ex)

1 validation error for Sphere
center
  List should have at least 2 items after validation, not 1 [type=too_short, input_value=(1,), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_short


As you can see the exception is much clearer.

Notice by the way how Pydantic coerced the tuple we specified for `center` into a list object.

In [11]:
from pydantic import PydanticUserError


try:
    class Model(BaseModel):
        unique_elements: conlist(int, min_length=1, max_length=10, unique_items=True)
except PydanticUserError as ex:
    print(ex)

`unique_items` is removed, use `Set` instead(this feature is discussed in https://github.com/pydantic/pydantic-core/issues/296)

For further information visit https://errors.pydantic.dev/2.5/u/removed-kwargs


If you look at Pydantic's documentation for `conlist`, you'll notice that it states there is an additional constrain we can place on lists: a uniqueness constraint that forces all the elements of the provided list to be unique.

The documentation states that this parameter has been deprecated (which means if you are writing new code you should **not** use it), and in fact as of the recording of this video, it has been removed! Let that serve as a warning to all of us not to use deprecated functionality!

(and, a pet peeve of mine, it is **deprecated**, not depreciated - depreciated is for accountants, deprecated is for developers!)

I do not agree with their recommendation to use `Set` - a set does contain unique elements only, but a set is not a sequence type - so, if we want a sequence type (such as a list) of unique elements, we'll have to find a different way (and we will, when we get to custom validators).